In [1]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re

%matplotlib inline

In [2]:
df_train = pd.read_csv('input/train.csv')
# df = df[pd.notnull(df['tags'])]
df_train.head(10)

,itemid,title,Category,image_path
0,307504,nyx sex bomb pallete natural palette,0,beauty_image/6b2e9cbb279ac95703348368aa65da09.jpg
1,461203,etude house precious mineral any cushion pearl...,1,beauty_image/20450222d857c9571ba8fa23bdedc8c9.jpg
2,3592295,milani rose powder blush,2,beauty_image/6a5962bed605a3dd6604ca3a4278a4f9.jpg
3,4460167,etude house baby sweet sugar powder,3,beauty_image/56987ae186e8a8e71fcc5a261ca485da.jpg
4,5853995,bedak revlon color stay aqua mineral make up,3,beauty_image/9c6968066ebab57588c2f757a240d8b9.jpg
5,6208490,dr pure whitening cream,4,beauty_image/77e6b7e9d5544adbfda6809b2351c4fa.jpg
6,6959546,chanel powder blush malice,2,beauty_image/485c4b8435a1e94976a569f3f014ea8b.jpg
7,8984461,snail white cream original 100,4,beauty_image/c7336f1c2e590d4c2bee219a6a0351a7.jpg
8,9184082,sunprise all proof spf 50,4,beauty_image/970b32aa659689f371516e5ceb423e25.jpg
9,10345338,eyebrow powder nyx satuan rp 15.000 pc,3,beauty_image/49a47745248031775c98593f70f7201a.jpg


In [3]:
import itertools
import os

%matplotlib inline

import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, TimeDistributed, Embedding
from keras.preprocessing import text, sequence
from keras import utils
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [4]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:1']

In [5]:
train_posts, test_posts, train_tags, test_tags = train_test_split(df_train['title'], 
                                                                  df_train['Category'], 
                                                                  test_size=0.3, 
                                                                  random_state=42)

In [6]:
df_train_subset = pd.read_csv('input/subset_training.csv')

In [35]:
max_words = 3000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

In [36]:
tokenize.fit_on_texts(df_train_subset['title']) # only fit on suset training
x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

In [39]:
encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [40]:
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [52]:
max_words = 50
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

is_class0 = df_train['Category']==0
class_0 = df_train[is_class0]


tokenize.fit_on_texts(class_0['title']) # only fit on subset data
x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)
 
for i in range(1,58):
    print (i)
    boolean_filter = df_train['Category']==i
    class_i = df_train[boolean_filter]
    
    max_words = 50
    tokenize = text.Tokenizer(num_words=max_words, char_level=False)
    
    tokenize.fit_on_texts(class_i['title'])
    x_train = np.append(x_train, tokenize.texts_to_matrix(train_posts), axis=1);
    x_test = np.append(x_test, tokenize.texts_to_matrix(test_posts), axis=1);

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57


In [53]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (466630, 2900)
x_test shape: (199985, 2900)
y_train shape: (466630, 58)
y_test shape: (199985, 58)


In [80]:
batch_size = 256
epochs = 50

In [81]:
x_train[1,:]

array([0., 0., 0., ..., 0., 0., 0.])

In [82]:
train_posts[1]

'etude house precious mineral any cushion pearl aura puff'

In [83]:
# Build the model
input_length = x_train.shape[1]
model = Sequential()
model.add(LSTM(1500,input_shape=(1,input_length)))
model.add(Dropout(0.6))
model.add(Dense(512, activation='elu'))
model.add(Dropout(0.6))
model.add(Dense(num_classes, activation='elu'))
model.add(Dropout(0.6))

model.add(Dense(num_classes, activation='elu'))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
#output_dir = 'lstm_search'
#tbCallBack = keras.callbacks.TensorBoard(log_dir=output_dir, histogram_freq=1, write_grads=True, write_graph= False, write_images=True)
#checkpoint = keras.callbacks.ModelCheckpoint(output_dir+'/best.h5', monitor='val_loss',save_best_only=True)

In [ ]:
history = model.fit(x_train.reshape((x_train.shape[0],1,x_train.shape[1])), y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 419967 samples, validate on 46663 samples
Epoch 1/50
419967/419967 [==============================] - 102s 242us/step - loss: 1.4248 - acc: 0.6071 - val_loss: 1.0668 - val_acc: 0.6764
Epoch 2/50
419967/419967 [==============================] - 93s 222us/step - loss: 1.2140 - acc: 0.6491 - val_loss: 1.0279 - val_acc: 0.6844
Epoch 3/50
419967/419967 [==============================] - 93s 222us/step - loss: 1.1732 - acc: 0.6588 - val_loss: 1.0067 - val_acc: 0.6884
Epoch 4/50
419967/419967 [==============================] - 93s 222us/step - loss: 1.1520 - acc: 0.6631 - val_loss: 1.0034 - val_acc: 0.6913
Epoch 5/50
419967/419967 [==============================] - 93s 222us/step - loss: 1.1338 - acc: 0.6675 - val_loss: 0.9888 - val_acc: 0.6957
Epoch 6/50
419967/419967 [==============================] - 93s 222us/step - loss: 1.1215 - acc: 0.6699 - val_loss: 0.9816 - val_acc: 0.6958
Epoch 7/50
419967/419967 [==============================] - 93s 222us/step - loss: 1.1084 - acc: 0.673

In [76]:
score = model.evaluate(x_test.reshape((x_test.shape[0],1,x_test.shape[1])), y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

199985/199985 [==============================] - 11s 57us/step
Test accuracy: 0.7105932944881459


In [77]:
from sklearn.metrics import classification_report
y_predicted = model.predict_classes(x_test.reshape((x_test.shape[0],1,x_test.shape[1])), verbose=1)

199985/199985 [==============================] - 44s 218us/step


In [78]:
y_test_num = np.argmax(y_test, axis=1)

In [79]:
print(classification_report(y_test_num, y_predicted))

              precision    recall  f1-score   support

           0       0.65      0.37      0.47      1108
           1       0.80      0.57      0.67      8587
           2       0.81      0.85      0.83      3446
           3       0.86      0.86      0.86     24217
           4       0.64      0.77      0.70     12714
           5       0.74      0.79      0.76     16764
           6       0.37      0.41      0.39       618
           7       0.77      0.71      0.74      3538
           8       0.57      0.64      0.61      1836
           9       0.77      0.41      0.53      2409
          10       0.70      0.54      0.61       321
          11       0.55      0.54      0.54      1204
          12       0.72      0.92      0.81      6535
          13       0.70      0.53      0.60       942
          14       0.45      0.04      0.08       766
          15       0.62      0.04      0.08       182
          16       0.93      0.02      0.04       668
          17       0.83    

In [31]:
df_public = pd.read_csv('input/test.csv')
df_public.head(10)

,itemid,title,image_path
0,370855998,flormar 7 white cream bb spf 30 40ml,beauty_image/1588591395c5a254bab84042005f2a9f.jpg
1,637234604,maybelline clear smooth all in one bb cream sp...,beauty_image/920985ed9587ea20f58686ea74e20f93.jpg
2,690282890,murah innisfree eco natural green tea bb cream...,beauty_image/90b40e5710f54352b243fcfb0f5d1d7f.jpg
3,930913462,loreal white perfect day cream spf 17 pa white...,beauty_image/289c668ef3d70e1d929d602d52d5d78a.jpg
4,1039280071,hada labo cc cream ultimate anti aging spf 35 ...,beauty_image/d5b3e652c5822d2306f4560488ec30c6.jpg
5,1327710392,cathy doll cc speed white powder pact spf 40 o...,beauty_image/e1e50828d5594721a7d5d5c1ff78afbd.jpg
6,1328802799,safi white natural brightening cream 45g,beauty_image/97ec852d5afc5d82ac02b80083cf292f.jpg
7,1330468145,light beige 03 bioaqua bb cushion exquisite de...,beauty_image/8ce1a5fe546f0cc795329bad599a8d5a.jpg
8,1677309730,new produk missha m perfect bb cream share in ...,beauty_image/755fcc85c687e8cb53d2a8d43ebfe251.jpg
9,1683142205,ready laneige bb cushion anti aging spf 50 pa,beauty_image/34b56398c099505c650cf2447dc9f21f.jpg


In [32]:
x_public = tokenize.texts_to_matrix(df_public["title"])

In [33]:
preds = model.predict(x_public.reshape((x_public.shape[0],1,x_public.shape[1])), verbose=1)

172402/172402 [==============================] - 32s 186us/step


In [34]:
df_public['Category'] = [np.argmax(pred) for pred in preds]
df_submit = df_public[['itemid', 'Category']].copy()
df_submit.to_csv('submission.csv', index=False)

In [38]:
model.save('Mar23_3pm.h5')